## Topic Modelling With LDA

In this notebook, we are gonna try to find related articles by using topic modeling for [COVID-19 Open Research Dataset](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) . I'm gonna use [LDA(Latent Dirichlet Allocation)](https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158). Shortly, LDA is being  acknowledged the most powerful topic modeling method. The main purpose of the LDA is to find a topics of a document belongs to, based on the words in it.  

### Explaratory Data Analysis

As a first step, we keep going with some  exploratory data analysis.

In [ ]:
#Import necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
from spacy import Lemmatizer
from gensim.corpora import Dictionary
%matplotlib inline